In [297]:
import plotly.offline as py
import plotly.graph_objs as go
import cufflinks as cf
import pandas as pd
import numpy as np

In [298]:
files = {
    'gun-violence': 'gun-violence.csv',
    'population': 'population-estimates.csv'
}

In [299]:
# create population data frame
pop_data = pd.read_csv(files['population'])
pop_data = pop_data.ix[2:]

In [300]:
# filter to relevant year
pop_data = pop_data[['GEO.display-label', 'respop72015']].set_index('GEO.display-label')
pop_data = pd.to_numeric(pop_data['respop72015'], errors='coerce')

In [301]:
# create data frame
data = pd.read_csv(files['gun-violence'])
data['incident_year'] = data['incident_date'].str[-2:]
data['mci'] = data['num_killed'] > 1
data['mci'] = np.where(data['num_killed']>1, 'y', 'n')

In [302]:
# group by state and mci / non-mci
plot_df = data.groupby(['state', 'mci'])['gva_id'].sum().unstack('mci')
plot_df['total_incidents'] = plot_df['n'] + plot_df['y']

In [303]:
# join data frames
plot_df = pd.concat([plot_df, pop_data], axis=1, join='inner').dropna()

In [304]:
plot_df['mci-per-capita'] = plot_df['y'].astype(float) / plot_df['respop72015']
plot_df['non-mci-per-capita'] = plot_df['n'].astype(float) / plot_df['respop72015']
plot_df['incidents_per_capita'] = plot_df['total_incidents'].astype(float) / plot_df['respop72015']

In [311]:
# sort by total incidents
plot_df = plot_df.sort_values('incidents_per_capita', ascending=False)

In [312]:
# Specify the y-position of the labels
y1 = plot_df['incidents_per_capita']
xcoord = range(0,len(plot_df.index))
annotations = [dict(
                x = xi,
                y = yi * 1.0005,
                text = str(int(round(yi,0))),
                yanchor = 'bottom',
                showarrow = False,
                font = dict(
                    size=9
                )
            ) for xi, yi in zip(xcoord, y1)]

In [313]:
trace1 = go.Bar(
    x = plot_df.index,
    y = plot_df['mci-per-capita'],
    name = 'Multi-Casualty Incidents'
)
trace2 = go.Bar(
    x = plot_df.index,
    y = plot_df['non-mci-per-capita'],
    name = 'Single-Casualty Incidents'
)
data = [trace1, trace2]
layout = go.Layout(
    barmode='stack',
    annotations=annotations
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='US Gun Violence')